In [1]:
import pandas as pd
from sqlalchemy import create_engine
from functions import list_countries_un


In [2]:
import pymysql
pymysql.install_as_MySQLdb()

<strong>Solar/Electricity: </strong>

In [3]:
un_solar_csv = "Resources/UNdata_total_electricity.csv"
df = pd.read_csv(un_solar_csv)
df.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Afghanistan,Electricity - Gross production,2017.0,"Kilowatt-hours, million",1098.2,1.0
1,Afghanistan,Electricity - Gross production,2016.0,"Kilowatt-hours, million",1076.3,1.0
2,Afghanistan,Electricity - Gross production,2015.0,"Kilowatt-hours, million",1033.7,1.0
3,Afghanistan,Electricity - Gross production,2014.0,"Kilowatt-hours, million",1049.3,1.0
4,Afghanistan,Electricity - Gross production,2013.0,"Kilowatt-hours, million",1022.3,1.0


In [4]:
electric_country_list = list_countries_un(df)

In [5]:
electric_country_list.head()

,country_name
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra


In [6]:
rds_connection_string = "root:64535864@localhost/energy_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [7]:
engine.table_names()

['EnergyConsumptionEIA',
 'EnergyConsumptionUN',
 'country',
 'eia_coal',
 'enerdata_oil',
 'un_solar_energy',
 'un_total_electric_energy',
 'us_state_electric']

In [8]:
electric_country_list.to_sql('country', con=engine, if_exists='append', index=False)

In [9]:
un_electic_df = df[['Country or Area', 'Commodity - Transaction', 'Year', 'Unit', 'Quantity']]

In [10]:
un_electic_df = un_electic_df.rename(columns={
    'Country or Area': 'country',
    'Commodity - Transaction': 'commodity_transaction',
    'Year': 'year',
    'Unit': 'unit',
    'Quantity': 'quantity'
})

In [11]:
un_electic_df = un_electic_df.dropna()

In [12]:
un_electic_df.to_sql(name='un_total_electric_energy', con=engine, if_exists='append', index=False)

In [49]:
eia_electric_csv = 'Resources/EIA_electricity_by_sector_by_state.csv'

In [50]:
df_electric = pd.read_csv(eia_electric_csv)

In [51]:
df_electric.head()

,"Table 5.4.B. Sales of Electricity to Ultimate Customers by End-Use Sector,",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,"by State, Year-to-Date through November 2018 a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Residential,NaN,Commercial,NaN,Industrial,NaN,Transportation,NaN,All Sectors,NaN
2,Census Division\nand State,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD,November 2018 YTD,November 2017 YTD
3,New England,"43,722","41,436","48,097","47,818","14,751","15,508",520,502,"107,090","105,264"
4,Connecticut,"11,882","11,141","11,391","11,313","2,836","2,993",178,162,"26,287","25,609"


In [ ]:
df_electric['country'] = 'United States'

In [ ]:
renamed_df_electric = df_electric.rename(columns={
    'Table 5.4.B. Sales of Electricity to Ultimate Customers by End-Use Sector,': 'division_state',
    'Unnamed: 1': 'resident_nov_2018',
    'Unnamed: 2': 'resident_nov_2017',
    'Unnamed: 3': 'comm_nov_2018',
    'Unnamed: 4': 'comm_nov_2017',
    'Unnamed: 5': 'indust_nov_2018',
    'Unnamed: 6': 'indust_nov_2017',
    'Unnamed: 7': 'trans_nov_2018',
    'Unnamed: 8': 'trans_nov_2017',
    'Unnamed: 9': 'all_sect_nov_2018',
    'Unnamed: 10': 'all_sect_nov_2017',
})

In [ ]:
renamed_df_electric.head()

In [ ]:
new_df_electric = renamed_df_electric[['country', 'division_state', 'resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']].iloc[3:]
new_df_electric.head()

In [ ]:
# method to replace ',' with ''
new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']] = new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']].apply(lambda x: x.str.replace(',',''))



In [ ]:
# convert to numeric
new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']] = new_df_electric[['resident_nov_2018','resident_nov_2017', 'comm_nov_2018', 'comm_nov_2017', 'indust_nov_2018', 'indust_nov_2017', 'trans_nov_2018', 'trans_nov_2017', 'all_sect_nov_2018', 'all_sect_nov_2017']].apply(pd.to_numeric)


In [ ]:
new_df_electric = new_df_electric.dropna()


In [ ]:
new_df_electric.to_sql(name='us_state_electric', con=engine, if_exists='append', index=False)

<strong>Natural Gas: </strong>

In [27]:
csv_file1 = "Resources/UNdata_Export_NaturalGasConsumption.csv"
Natural_Gas_Consumption_UN_df = pd.read_csv(csv_file1)
Natural_Gas_Consumption_UN_df.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Afghanistan,Natural gas (including LNG) - final consumption,2017.0,Terajoules,6091.1781,1.0
1,Afghanistan,Natural gas (including LNG) - final consumption,2016.0,Terajoules,6450.1713,1.0
2,Afghanistan,Natural gas (including LNG) - final consumption,2015.0,Terajoules,5704.8702,1.0
3,Afghanistan,Natural gas (including LNG) - final consumption,2014.0,Terajoules,5537.0799,1.0
4,Afghanistan,Natural gas (including LNG) - final consumption,2013.0,Terajoules,6028.7445,1.0


In [28]:
natural_gas_country_list = list_countries_un(Natural_Gas_Consumption_UN_df)
natural_gas_country_list.count()

country_name    31
dtype: int64

In [29]:
natural_gas_country_list.to_sql('country', con=engine, if_exists='append', index=False)

In [30]:
#Drop Column 'Quantity Footnotes'
Natural_Gas_Consumption_UN_df.drop(Natural_Gas_Consumption_UN_df.columns[[5]], axis=1, inplace=True)
Natural_Gas_Consumption_UN_df.head()

,Country or Area,Commodity - Transaction,Year,Unit,Quantity
0,Afghanistan,Natural gas (including LNG) - final consumption,2017.0,Terajoules,6091.1781
1,Afghanistan,Natural gas (including LNG) - final consumption,2016.0,Terajoules,6450.1713
2,Afghanistan,Natural gas (including LNG) - final consumption,2015.0,Terajoules,5704.8702
3,Afghanistan,Natural gas (including LNG) - final consumption,2014.0,Terajoules,5537.0799
4,Afghanistan,Natural gas (including LNG) - final consumption,2013.0,Terajoules,6028.7445


In [31]:
#Rename columns
Natural_Gas_Consumption_UN_df.columns = ['Country', 'Commodity_Transaction', 'Year', 'Unit', 'Quantity']
Natural_Gas_Consumption_UN_df.head()

,Country,Commodity_Transaction,Year,Unit,Quantity
0,Afghanistan,Natural gas (including LNG) - final consumption,2017.0,Terajoules,6091.1781
1,Afghanistan,Natural gas (including LNG) - final consumption,2016.0,Terajoules,6450.1713
2,Afghanistan,Natural gas (including LNG) - final consumption,2015.0,Terajoules,5704.8702
3,Afghanistan,Natural gas (including LNG) - final consumption,2014.0,Terajoules,5537.0799
4,Afghanistan,Natural gas (including LNG) - final consumption,2013.0,Terajoules,6028.7445


In [32]:
engine.table_names()

['EnergyConsumptionEIA',
 'EnergyConsumptionUN',
 'country',
 'eia_coal',
 'enerdata_oil',
 'un_solar_energy',
 'un_total_electric_energy',
 'us_state_electric']

In [33]:
Natural_Gas_Consumption_UN_df.to_sql(name='EnergyConsumptionUN', con=engine, if_exists='append', index=False)

In [34]:
csv_file1 = "Resources/Total_consumption_Natural_Gas_EIA2.csv"
Natural_Gas_Consumption_EIA_df = pd.read_csv(csv_file1)
Natural_Gas_Consumption_EIA_df.head()

<<<<<<< HEAD
NaN            NaN      1990.000000 1991.000000 1992.000000 1993.000000 1994.000000 1995.00000 1996.00000 1997.000000 1998.000000 1999.000000 2000.000000 2001.000000 2002.000000 2003.000000 2004.000000 2005.000000 2006.000000 2007.00000 2008.000000 2009.000000 2010.000000 2011.000000 2012.000000 2013.000000 2014.000000 2015.000000   2016.000000
Primary Energy Quad Btu NaN         NaN         NaN         NaN         NaN         NaN        NaN        NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN        NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN                   NaN
Production     NaN      NaN         NaN         NaN         NaN         NaN         NaN        NaN        NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN        NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN                   NaN
                                                                                                                                                                                                                                                                                                                                 NaN                   NaN
Afghanistan    Quad Btu 0.118801    0.019883    0.016393    0.016307    0.016231    0.01223    0.01186    0.011376    0.011147    0.011116    0.011231    0.007873    0.008175    0.009243    0.008484    0.008685    0.009183    0.01397    0.013774    0.018586    0.027522    0.041304    0.036962    0.039703    0.044375    0.042368         0.049234

In [35]:
#Delete Country = Production rows
Natural_Gas_Consumption_EIA_df = Natural_Gas_Consumption_EIA_df.dropna()
Natural_Gas_Consumption_EIA_df.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,<<<<<<< HEAD
NaN,NaN,1990.000000,1991.000000,1992.000000,1993.000000,1994.000000,1995.000000,1996.000000,1997.000000,1998.000000,1999.000000,2000.000000,2001.000000,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000
Afghanistan,Quad Btu,0.118801,0.019883,0.016393,0.016307,0.016231,0.012230,0.011860,0.011376,0.011147,0.011116,0.011231,0.007873,0.008175,0.009243,0.008484,0.008685,0.009183,0.013970,0.013774,0.018586,0.027522,0.041304,0.036962,0.039703,0.044375,0.042368,0.049234
Albania,Quad Btu,0.127906,0.095521,0.066078,0.062657,0.066505,0.068651,0.082783,0.074230,0.066803,0.068543,0.061270,0.051361,0.051141,0.070020,0.072381,0.071455,0.068797,0.043395,0.053013,0.064617,0.100664,0.075430,0.084966,0.107084,0.093596,0.102940,0.120062
Algeria,Quad Btu,4.752963,4.916415,5.032649,4.884628,4.735501,5.048409,5.248275,5.592483,5.749333,6.020038,6.190591,6.169600,6.390536,6.788416,6.900755,7.479914,7.417509,7.129999,7.179031,6.944566,6.928997,6.830145,7.001792,6.512557,6.564005,6.698295,6.912613
American Samoa,Quad Btu,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
Natural_Gas_Consumption_EIA_df.columns = ['Year_' + str(col) for col in Natural_Gas_Consumption_EIA_df.columns]
Natural_Gas_Consumption_EIA_df.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,Year_<<<<<<< HEAD
NaN,NaN,1990.000000,1991.000000,1992.000000,1993.000000,1994.000000,1995.000000,1996.000000,1997.000000,1998.000000,1999.000000,2000.000000,2001.000000,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000
Afghanistan,Quad Btu,0.118801,0.019883,0.016393,0.016307,0.016231,0.012230,0.011860,0.011376,0.011147,0.011116,0.011231,0.007873,0.008175,0.009243,0.008484,0.008685,0.009183,0.013970,0.013774,0.018586,0.027522,0.041304,0.036962,0.039703,0.044375,0.042368,0.049234
Albania,Quad Btu,0.127906,0.095521,0.066078,0.062657,0.066505,0.068651,0.082783,0.074230,0.066803,0.068543,0.061270,0.051361,0.051141,0.070020,0.072381,0.071455,0.068797,0.043395,0.053013,0.064617,0.100664,0.075430,0.084966,0.107084,0.093596,0.102940,0.120062
Algeria,Quad Btu,4.752963,4.916415,5.032649,4.884628,4.735501,5.048409,5.248275,5.592483,5.749333,6.020038,6.190591,6.169600,6.390536,6.788416,6.900755,7.479914,7.417509,7.129999,7.179031,6.944566,6.928997,6.830145,7.001792,6.512557,6.564005,6.698295,6.912613
American Samoa,Quad Btu,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
engine.table_names()

['EnergyConsumptionEIA',
 'EnergyConsumptionUN',
 'country',
 'eia_coal',
 'enerdata_oil',
 'un_solar_energy',
 'un_total_electric_energy',
 'us_state_electric']

In [38]:
#Use pandas to load csv concerted to dataFrame into database
Natural_Gas_Consumption_EIA_df.to_sql(name='EnergyConsumptionEIA', con=engine, if_exists='append', index=False)

InternalError: (pymysql.err.InternalError) (1054, "Unknown column 'Year_<<<<<<< HEAD' in 'field list'") [SQL: 'INSERT INTO `EnergyConsumptionEIA` (`Year_<<<<<<< HEAD`) VALUES (%(Year_<<<<<<< HEAD)s)'] [parameters: ({'Year_<<<<<<< HEAD': 2016.0}, {'Year_<<<<<<< HEAD': 0.049233547999999995}, {'Year_<<<<<<< HEAD': 0.120062319}, {'Year_<<<<<<< HEAD': 6.912613307999999}, {'Year_<<<<<<< HEAD': 0.0}, {'Year_<<<<<<< HEAD': 3.9351578560000005}, {'Year_<<<<<<< HEAD': 9.23e-06}, {'Year_<<<<<<< HEAD': 5.539999999999999e-05}  ... displaying 10 of 893 total bound parameter sets ...  {'Year_<<<<<<< HEAD': 0.15683357}, {'Year_<<<<<<< HEAD': 0.164709569})] (Background on this error at: http://sqlalche.me/e/2j85)

<strong>Oil</strong>

In [39]:
# Enerdata data
Enerdata_oil ="Resources/Enerdata World Oil Consumption.csv"
Enerdata_oil_data = pd.read_csv(Enerdata_oil)
Enerdata_oil_data.head()

,Oil products domestic consumption (Mt),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Column1,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2016 - 2017 (%),2000 - 2017 (%/year)
2,World,3090.0,3116.0,3132.0,3132.0,3164.0,3236.0,3317.0,3395.0,3426.0,...,3908.0,3938.0,3970.0,4007.0,4050.0,4121.0,4188.0,4258.0,1.7,1.1
3,OECD,1789.0,1800.0,1840.0,1855.0,1905.0,1919.0,1966.0,1992.0,2001.0,...,1893.0,1861.0,1834.0,1817.0,1800.0,1822.0,1844.0,1864.0,1.1,-0.5
4,G7,1387.0,1381.0,1403.0,1409.0,1435.0,1439.0,1472.0,1485.0,1498.0,...,1365.0,1344.0,1320.0,1309.0,1300.0,1308.0,1312.0,1323.0,0.8,-0.8


In [40]:
# EIA data
EIA_coal ="Resources/EIA_International_Data.csv"
EIA_coal_data = pd.read_csv(EIA_coal)
EIA_coal_data.head()

,Unnamed: 0,Unnamed: 1,1990,1991,1992,1993,1994,1995,1996,1997,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Afghanistan,Quad Btu,0.002187572,0.001958398,0.000166672,0.000145838,0.000125004,0.00010417,6.25E-05,4.17E-05,...,0.005062667,0.007229405,0.010417009,0.015104664,0.03083431,0.025834152,0.028063389,0.031613501,0.028438401,0.03537612
1,Albania,Quad Btu,0.057443992,0.032323962,0.008462611,0.005516766,0.00452589,0.002142433,0.003026187,0.001044436,...,0.001446142,0.001874629,0.004311647,0.004740134,0.00581135,0.006775445,0.002812672,0.003558197,0.003023455,0.002352831
2,Algeria,Quad Btu,0.028251904,0.031484709,0.029095244,0.022629634,0.022966971,0.013493447,0.008939408,0.01523635,...,0.015373496,0.01567704,0.003821761,7.96E-05,5.65E-05,0.00033495,0.001109634,0.001079212,0.000698246,0.000697224
3,American Samoa,Quad Btu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angola,Quad Btu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# Enerdata Data Transformation:

# 1) Rename/Organize Columns (years / floattypes)
oil_rename = Enerdata_oil_data.rename(columns={
    'Oil products domestic consumption (Mt)': 'Country',
    'Unnamed: 1' : 'Year_1990',
    'Unnamed: 2' : 'Year_1991',
    'Unnamed: 3' : 'Year_1992',
    'Unnamed: 4' : 'Year_1993',
    'Unnamed: 5' : 'Year_1994',
    'Unnamed: 6' : 'Year_1995',
    'Unnamed: 7' : 'Year_1996',
    'Unnamed: 8' : 'Year_1997',
    'Unnamed: 9' : 'Year_1998',
    'Unnamed: 10' : 'Year_1999',
    'Unnamed: 11' : 'Year_2000',
    'Unnamed: 12' : 'Year_2001',
    'Unnamed: 13' : 'Year_2002',
    'Unnamed: 14' : 'Year_2003',
    'Unnamed: 15' : 'Year_2004',
    'Unnamed: 16' : 'Year_2005',
    'Unnamed: 17' : 'Year_2006',
    'Unnamed: 18' : 'Year_2007',
    'Unnamed: 19' : 'Year_2008',
    'Unnamed: 20' : 'Year_2009',
    'Unnamed: 21' : 'Year_2010',
    'Unnamed: 22' : 'Year_2011',
    'Unnamed: 23' : 'Year_2012',
    'Unnamed: 24' : 'Year_2013',
    'Unnamed: 25' : 'Year_2014',
    'Unnamed: 26' : 'Year_2015',
    'Unnamed: 27' : 'Year_2016',
    'Unnamed: 28' : 'Year_2017'
})

# 2) Drop unnecessary COLUMNS
oil_drop = oil_rename.drop(columns={'Unnamed: 29', 'Unnamed: 30'})

# 3) Drop Unneccessary/Repetitive ROWS
    #oil_drop2 = oil_drop.drop([0,1,2,3,4,5,6, 60,61,62], axis=0)
oil_drop2 = oil_drop.drop([0,1], axis=0)

# oil_drop.reset_index=()

# 4) Rename/Print new DataFrame
Enerdata_oil_consumption_df = oil_drop2
Enerdata_oil_consumption_df.head()

,Country,Year_1990,Year_1991,Year_1992,Year_1993,Year_1994,Year_1995,Year_1996,Year_1997,Year_1998,...,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017
2,World,3090.0,3116.0,3132.0,3132.0,3164.0,3236.0,3317.0,3395.0,3426.0,...,3898.0,3804.0,3908.0,3938.0,3970.0,4007.0,4050.0,4121.0,4188.0,4258.0
3,OECD,1789.0,1800.0,1840.0,1855.0,1905.0,1919.0,1966.0,1992.0,2001.0,...,1959.0,1866.0,1893.0,1861.0,1834.0,1817.0,1800.0,1822.0,1844.0,1864.0
4,G7,1387.0,1381.0,1403.0,1409.0,1435.0,1439.0,1472.0,1485.0,1498.0,...,1420.0,1343.0,1365.0,1344.0,1320.0,1309.0,1300.0,1308.0,1312.0,1323.0
5,BRICS,486.0,506.0,494.0,489.0,460.0,487.0,495.0,517.0,526.0,...,765.0,766.0,802.0,847.0,884.0,912.0,950.0,992.0,1018.0,1058.0
6,Europe,649.0,655.0,653.0,648.0,651.0,662.0,680.0,680.0,689.0,...,650.0,616.0,612.0,586.0,569.0,556.0,549.0,564.0,574.0,587.0


In [42]:
# EIA Data Transformation

# 1) Rename/Organize Columns (years / floattypes)
coal_rename = EIA_coal_data.rename(columns={
    'Unnamed: 0' : 'Country',
    '1990' : 'Year_1990', 
    '1991' : 'Year_1991',
    '1992' : 'Year_1992',
    '1993' : 'Year_1993',
    '1994' : 'Year_1994',
    '1995' : 'Year_1995', 
    '1996' : 'Year_1996',
    '1997' : 'Year_1997',
    '1998' : 'Year_1998',
    '1999' : 'Year_1999', 
    '2000' : 'Year_2000',
    '2001' : 'Year_2001',
    '2002' : 'Year_2002',
    '2003' : 'Year_2003',
    '2004' : 'Year_2004',
    '2005' : 'Year_2005',
    '2006' : 'Year_2006',
    '2007' : 'Year_2007',
    '2008' : 'Year_2008',
    '2009' : 'Year_2009',
    '2010' : 'Year_2010',
    '2011' : 'Year_2011',
    '2012' : 'Year_2012',
    '2013' : 'Year_2013',
    '2014' : 'Year_2014',
    '2015' : 'Year_2015',
    '2016' : 'Year_2016'
    
})
coal_rename


# 2) Drop Unnecessary COLUMNS (Quad Btu)
coal_drop = coal_rename.drop(columns={"Unnamed: 1"})
#coal_drop.reset_index=()
#coal_drop

# 3) Rename/Print new DataFrame
EIA_coal_consumption_df = coal_drop.dropna()
EIA_coal_consumption_df.head(10)

,Country,Year_1990,Year_1991,Year_1992,Year_1993,Year_1994,Year_1995,Year_1996,Year_1997,Year_1998,...,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016
0,Afghanistan,0.002187572,0.001958398,0.000166672,0.000145838,0.000125004,0.00010417,6.25E-05,4.17E-05,4.17E-05,...,0.005062667,0.007229405,0.010417009,0.015104664,0.03083431,0.025834152,0.028063389,0.031613501,0.028438401,0.03537612
1,Albania,0.057443992,0.032323962,0.008462611,0.005516766,0.00452589,0.002142433,0.003026187,0.001044436,0.00131224,...,0.001446142,0.001874629,0.004311647,0.004740134,0.00581135,0.006775445,0.002812672,0.003558197,0.003023455,0.002352831
2,Algeria,0.028251904,0.031484709,0.029095244,0.022629634,0.022966971,0.013493447,0.008939408,0.01523635,0.017653926,...,0.015373496,0.01567704,0.003821761,7.96E-05,5.65E-05,0.00033495,0.001109634,0.001079212,0.000698246,0.000697224
3,American Samoa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Angola,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Antarctica,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Antigua and Barbuda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Argentina,0.037816345,0.033326859,0.033271773,0.030903087,0.044839774,0.039771888,0.036824802,0.033877716,0.034428573,...,0.038945602,0.041011316,0.033602287,0.041424459,0.048998745,0.044536802,0.048337717,0.046411572,0.043798336,0.035270629
8,Armenia,--,--,0.002846723,6.06E-05,0.000726823,6.06E-05,0.000100948,0.000100948,0.000100948,...,6.06E-05,4.04E-05,0,2.02E-05,0,4.04E-05,2.74E-05,2.35E-05,2.61E-05,2.72E-05
9,Aruba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
EIA_coal_consumption_df = EIA_coal_consumption_df.replace(to_replace={'--': None})

In [44]:
EIA_coal_consumption_df[[
    'Year_1990',
    'Year_1991',
    'Year_1992',
    'Year_1993',
    'Year_1994',
    'Year_1995',
    'Year_1996',
    'Year_1997',
    'Year_1998',
    'Year_1999',
    'Year_2000',
    'Year_2001',
    'Year_2002',
    'Year_2003',
    'Year_2004',
    'Year_2005',
    'Year_2006',
    'Year_2007',
    'Year_2008',
    'Year_2009',
    'Year_2010',
    'Year_2011',
    'Year_2012',
    'Year_2013',
    'Year_2014',
    'Year_2015',
    'Year_2016'
]].apply(pd.to_numeric)

,Year_1990,Year_1991,Year_1992,Year_1993,Year_1994,Year_1995,Year_1996,Year_1997,Year_1998,Year_1999,...,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016
0,0.002188,0.001958,0.000167,0.000146,0.000125,0.000104,0.000063,0.000042,0.000042,0.000021,...,5.062667e-03,0.007229,0.010417,0.015105,0.030834,0.025834,0.028063,0.031614,0.028438,0.035376
1,0.057444,0.032324,0.008463,0.005517,0.004526,0.002142,0.003026,0.001044,0.001312,0.003214,...,1.446142e-03,0.001875,0.004312,0.004740,0.005811,0.006775,0.002813,0.003558,0.003023,0.002353
2,0.028252,0.031485,0.029095,0.022630,0.022967,0.013493,0.008939,0.015236,0.017654,0.017766,...,1.537350e-02,0.015677,0.003822,0.000080,0.000056,0.000335,0.001110,0.001079,0.000698,0.000697
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.037816,0.033327,0.033272,0.030903,0.044840,0.039772,0.036825,0.033878,0.034429,0.036549,...,3.894560e-02,0.041011,0.033602,0.041424,0.048999,0.044537,0.048338,0.046412,0.043798,0.035271
8,NaN,NaN,0.002847,0.000061,0.000727,0.000061,0.000101,0.000101,0.000101,0.000061,...,6.060000e-05,0.000040,0.000000,0.000020,0.000000,0.000040,0.000027,0.000023,0.000026,0.000027
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [45]:
# Confirm Tables
engine.table_names()

['EnergyConsumptionEIA',
 'EnergyConsumptionUN',
 'country',
 'eia_coal',
 'enerdata_oil',
 'un_solar_energy',
 'un_total_electric_energy',
 'us_state_electric']

In [46]:
Enerdata_oil_consumption_df.to_sql(name='enerdata_oil', con=engine, if_exists='append', index=False)

In [47]:
EIA_coal_consumption_df.to_sql(name='eia_coal', con=engine, if_exists='append', index=False)